# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [54]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [55]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.reset_index(drop=True)
city_data_df.head()

,City,Latitude,Longitude,Max Temp (C),Humidity (g per kg),Cloudiness (%),Wind Speed (m per s),Country,Date
City_ID,,,,,,,,,
0,Barcelos,41.5388,-8.6151,18.16,80,0,3.35,PT,2023-07-25 17:28:18
1,Hermanus,-34.4187,19.2345,7.67,68,0,2.38,ZA,2023-07-25 17:24:13
2,Adamstown,-25.0660,-130.1015,22.54,91,100,17.51,PN,2023-07-25 17:28:19
3,Tamanrasset,22.7850,5.5228,30.95,11,0,2.06,DZ,2023-07-25 17:28:19
4,Dudinka,69.4058,86.1778,14.79,62,0,4.96,RU,2023-07-25 17:28:19


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [56]:
%%capture --no-display

# Configure the map plot
cities_map_plot = city_data_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity (g per kg)",
    scale = 1,
    color = "City"
)

# Display the map
cities_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity (g per kg))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [57]:
# Narrow down cities that fit criteria and drop any results with null values
# We want max temp between 21 and 27 degrees, a gentle breeze (1.5-5 m/s), 
#and some shade but not much (5-15% cloud cover)
ideal_df = city_data_df.loc[city_data_df["Max Temp (C)"] <= 27 ]
ideal_df = ideal_df.loc[ideal_df["Max Temp (C)"] >= 21]
ideal_df = ideal_df.loc[ideal_df["Wind Speed (m per s)"]>=1.5]
ideal_df = ideal_df.loc[ideal_df["Wind Speed (m per s)"]<5]
ideal_df = ideal_df.loc[ideal_df["Cloudiness (%)"]>=3]
ideal_df = ideal_df.loc[ideal_df["Cloudiness (%)"]<15]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df

,City,Latitude,Longitude,Max Temp (C),Humidity (g per kg),Cloudiness (%),Wind Speed (m per s),Country,Date
City_ID,,,,,,,,,
163,Bengkulu,-3.8004,102.2655,23.66,90,8,1.80,ID,2023-07-25 17:24:33
243,Curvelo,-18.7564,-44.4308,23.38,33,10,2.24,BR,2023-07-25 17:27:48
282,Aydarken,39.9443,71.3429,21.48,40,4,2.71,KG,2023-07-25 17:29:44
467,Anloga,5.7947,0.8973,24.40,93,10,2.86,GH,2023-07-25 17:30:36
515,Jatai,-17.8814,-51.7144,22.36,49,9,3.50,BR,2023-07-25 17:30:45


### Step 3: Create a new DataFrame called `hotel_df`.

In [58]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=''

# Display sample data
hotel_df

,City,Latitude,Longitude,Max Temp (C),Humidity (g per kg),Cloudiness (%),Wind Speed (m per s),Country,Date,Hotel Name
City_ID,,,,,,,,,,
163,Bengkulu,-3.8004,102.2655,23.66,90,8,1.80,ID,2023-07-25 17:24:33,
243,Curvelo,-18.7564,-44.4308,23.38,33,10,2.24,BR,2023-07-25 17:27:48,
282,Aydarken,39.9443,71.3429,21.48,40,4,2.71,KG,2023-07-25 17:29:44,
467,Anloga,5.7947,0.8973,24.40,93,10,2.86,GH,2023-07-25 17:30:36,
515,Jatai,-17.8814,-51.7144,22.36,49,9,3.50,BR,2023-07-25 17:30:45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [59]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "language":"en"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    #Find the latituded and longitude:
    lat = hotel_df["Latitude"][index]
    lon = hotel_df["Longitude"][index]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
Bengkulu - nearest hotel: Dena Hotel
Curvelo - nearest hotel: Hotel Bandeirante
Aydarken - nearest hotel: No hotel found
Anloga - nearest hotel: Hotel de White House
Jatai - nearest hotel: Hotel Rio Claro


,City,Latitude,Longitude,Max Temp (C),Humidity (g per kg),Cloudiness (%),Wind Speed (m per s),Country,Date,Hotel Name
City_ID,,,,,,,,,,
163,Bengkulu,-3.8004,102.2655,23.66,90,8,1.80,ID,2023-07-25 17:24:33,Dena Hotel
243,Curvelo,-18.7564,-44.4308,23.38,33,10,2.24,BR,2023-07-25 17:27:48,Hotel Bandeirante
282,Aydarken,39.9443,71.3429,21.48,40,4,2.71,KG,2023-07-25 17:29:44,No hotel found
467,Anloga,5.7947,0.8973,24.40,93,10,2.86,GH,2023-07-25 17:30:36,Hotel de White House
515,Jatai,-17.8814,-51.7144,22.36,49,9,3.50,BR,2023-07-25 17:30:45,Hotel Rio Claro


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [60]:
%%capture --no-display

# Configure the map plot
hotels_map_plot = hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity (g per kg)",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotels_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity (g per kg),Hotel Name,Country)